In [95]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import time
import math

In [96]:
def get_price(bs_flat):
    return re.sub(r'\xa0','', bs_flat.find('span', attrs={
                'itemprop':'price'
            }).attrs['content']).replace('₽/мес.','')


In [97]:
def get_total_offers(soup):
    text = soup.find('div', attrs={'class': re.compile('.*totalOffers.*')}).text
    return int(''.join(re.findall(r'\d+', text)))

In [98]:
def get_total_pages(total_offers):
    return math.ceil(total_offers/28)

In [99]:
class main_page_parser:
    
    def __init__(self, url, headers):
        self.url = url
        self.headers  = headers
        self.soup = self.get_soup()
        self.flat_ids = self.get_flat_ids()
        self.href, self.price = self.get_page_price_href()
        
    def get_soup(self):
        resp = requests.get(self.url, headers=self.headers)
        soup = BeautifulSoup(resp.text)
        return soup
    
    def get_flat_ids(self):
        room_url = re.compile('https://www.cian.ru/rent/flat/[0-9]*')
        container = re.compile('.*--info-container--.*')
        info_containers = self.soup.find_all('div', attrs={
                        'class':container
                    })
        flat_ids = []
        for container in info_containers:
            href = container.find('a', {'href': room_url,
                                        'class': re.compile('.*--header--.*')
                                       }
                                 ).attrs['href']
            flat_id = re.search(r'[0-9]+', href).group()
            flat_ids.append(flat_id)
        return flat_ids
    
    def get_page_price_href(self):
        room_url = re.compile('https://www.cian.ru/rent/flat/[0-9]*')
        container = re.compile('.*--info-container--.*')
        info_containers = self.soup.find_all('div', attrs={
                        'class':container
                    })
        href_dict = {}
        price_dict = {}
        for container in info_containers:
            href = container.find('a', {'href': room_url,
                                        'class': re.compile('.*--header--.*')
                                       }
                                 ).attrs['href']
                        
            price = container.find('div', re.compile('.*--header--.*')).text.replace('₽/мес.','').replace(' ','')
            flat_id = re.search(r'[0-9]+', href).group()
            href_dict[flat_id] = href
            price_dict[flat_id] = price
        return href_dict, price_dict

In [134]:
class flat_page_parser:
    
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        self.soup = self.get_soup()
        self.more_price_info = self.get_more_price_info()
        self.main_info = self.get_main_info()
        self.address = self.get_address()
        self.undergrounds = self.get_undergrounds()
        self.house = self.house_properties()
        self.flat_attrs = self.get_flat_attrs()
        self.flat_properties = self.get_flat_properties()
        self.short_description = self.get_short_description()
        self.main_description = self.get_main_description()
        
    def get_soup(self):
        request = requests.get(self.url)
        soup = BeautifulSoup(request.text)
        return soup
    
    def get_more_price_info(self):
        dop_info = re.compile(r'.*more_price_rent.*')
        return re.sub(r'\xa0', ' ', self.soup.find('div', attrs={
                            'class': dop_info
                        }).text)

    def get_main_info(self):
        more_info = re.compile(r'.*--info-block--.*')
        info = self.soup.find('div', attrs={
            'class': more_info
        })

        info_re = re.compile(r'.*--info--.*')
        info_title = re.compile(r'.*--info-title--.*')
        info_text = re.compile(r'.*--info-text--.*')


        all = info.find_all('div', attrs={
                    'class': info_re
                })

        title_text={}
        for inform in all:
            key = inform.find('div', attrs={
                    'class': info_title
                }).text

            val = inform.find('div', attrs={
                    'class': info_text
                }).text
            title_text[key] = val

        return title_text
    
    def get_address(self):
        address = re.compile(r'.*--geo--.*')
        geo = self.soup.find('div', attrs={'class': address})
        tag = geo.find('span', attrs={'itemprop': 'name'}) 
        return tag['content']
    
    def get_undergrounds(self):
        undergrounds = re.compile(r'.*--underground--.*')

        all_undergrounds = self.soup.find_all('li', attrs={
            'class': undergrounds
        })

        und_str = str()
        for und in all_undergrounds:
            und_link_reg = re.compile(r'.*--underground_link--.*')
            und_time_reg = re.compile(r'.*--underground_time--.*')

            und_name = und.find('a', attrs={
                'class': und_link_reg
            }).text
            und_time = und.find('span', attrs={
                'class': und_time_reg
            })
            
            if und_time is None:
                und_time = ''
            else:
                und_time = und_time.text[4:]
            
            und_str += und_name + ' ' + und_time + ' '

        return und_str
    
    def house_properties(self):
        house_dict={}
        for item in self.soup.find_all('div', attrs= {'class': re.compile(r'.*--item--.*')}):
            house_attr_name = item.find('div', attrs = {'class': re.compile('.*--name--.*')}).text
            house_attr_value = item.find('div', attrs = {'class': re.compile('.*--value--.*')}).text
            house_dict[house_attr_name] = house_attr_value
        return house_dict
    
    def get_flat_attrs(self):
        """
        Return list of values: [Можно с детьми, можно с животными]
        """
        tag_info = self.soup.find('div', attrs={'class': re.compile('.*--section_divider--.*')})
        tag_sections = tag_info.find_all('ul', attrs={'class': re.compile('.*--container--.*')})
        atrib = []
        for section in tag_sections:
            for x in section.find_all('li', attrs={'class': re.compile('.*--item--.*')}):
                atrib.append(x.text)
        return atrib
    
    def get_flat_properties(self):
    
        """
        Return dict of values: {Тип жилья: ..., Планировка:...}
        """
        dop_attrs = {}
        container = self.soup.find('article', re.compile('.*--container--.*'))
        item = container.find_all('li', re.compile('.*--item.*--'))
        for it in item:
            name = it.find('span', re.compile('.*--name--.*')).text
            value = it.find('span', re.compile('.*--value--.*')).text
            dop_attrs[name] = value
        return dop_attrs
    
    def get_short_description(self):
        terms = self.soup.find('div', attrs={'class':re.compile('.*terms.*')})
        return re.sub(r'\xa0', ' ',  terms.find('p', attrs={'class': re.compile('.*description.*')}).text)
    
    def get_main_description(self):
        return self.soup.find('p', attrs={'itemprop': 'description'}).text

In [ ]:
m_p = main_page_parser(url = r'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=1&region=1&type=4',
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
          'Sec-Fetch-Mode': 'cors' })

In [118]:
f_p = flat_page_parser(url = r'https://www.cian.ru/rent/flat/215439272/',
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
          'Sec-Fetch-Mode': 'cors' })

In [122]:
f_p.address['content']

'Москва, ЦАО, р-н Пресненский, наб. Пресненская, 8с1'

In [ ]:
url = 'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=1&region=1&type=4'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
          'Sec-Fetch-Mode': 'cors' }

first_request = requests.get(url, headers=headers)
soup = BeautifulSoup(first_request.text)

In [136]:
total_offers = get_total_offers(soup)

In [137]:
total_pages = get_total_pages(total_offers)

In [135]:
%%time
total_pages = 20
flat_df = pd.DataFrame()
for page_num in tqdm(range(10, total_pages+1)):
    url = r'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={}&region=1&type=4'.format(page_num)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
          'Sec-Fetch-Mode': 'cors' }
    mpp = main_page_parser(url, headers)
    
    for item in mpp.price.items():
        if item[0] not in flat_df.index:
            flat_df.loc[item[0], 'price'] = item[1]
            flat_df.loc[item[0], 'href'] = mpp.href[item[0]]
            
            fpp = flat_page_parser(mpp.href[item[0]], headers=headers)
            flat_df.loc[item[0], 'more_price_info'] = fpp.more_price_info
            
            for info in fpp.main_info.items():
                flat_df.loc[item[0], info[0]] = info[1]
            
            flat_df.loc[item[0], 'address'] = fpp.address
            
            flat_df.loc[item[0], 'undergrounds'] = fpp.undergrounds
            
            for hp in fpp.house.items():
                flat_df.loc[item[0], hp[0]] = hp[1]
            
            for flat_atr in fpp.flat_attrs:
                flat_df.loc[item[0], flat_atr] = 1
            
            for flat_prp in fpp.flat_properties.items():
                flat_df.loc[item[0], flat_prp[0]] = flat_prp[1]
                
            flat_df.loc[item[0], 'short_description'] = fpp.short_description
            flat_df.loc[item[0], 'main_description'] = fpp.main_description
            
        elif item[1] != flat_df.loc[item[0], 'price']:
            flat_df.loc[item[0], 'price'] = item[1]
            flat_df.loc[itemp[0], 'change_price'] = 1
            

Wall time: 3min 59s


In [138]:
total_pages

860

In [140]:
flat_df

,price,href,more_price_info,Общая,Жилая,Кухня,Этаж,address,undergrounds,Лифты,...,Газоснабжение,Посудомоечная машина,Ванна,Планировка,Ванная комната,Балкон/лоджия,Вид из окон,Построен,Мусоропровод,Высота потолков
209074866,55000,https://www.cian.ru/rent/flat/209074866/,Комм. платежи включены (без счётчиков),42 м²,32 м²,10 м²,3 из 6,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",ЗИЛ 12 мин. пешком Автозаводская 15 мин. пешко...,Нет,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213955093,65000,https://www.cian.ru/rent/flat/213955093/,Комм. платежи включены (без счётчиков),45 м²,34 м²,5 м²,3 из 9,"Москва, ЮАО, р-н Донской, ул. Орджоникидзе, 6/9",Ленинский проспект 9 мин. пешком Площадь Гагар...,1 пассажирский,...,Центральное,1.0,1.0,Смежная,Есть,1 балкон,Во двор,NaN,NaN,NaN
212894519,75000,https://www.cian.ru/rent/flat/212894519/,Комм. платежи включены (без счётчиков),36 м²,18 м²,5 м²,9 из 9,"Москва, ЦАО, р-н Пресненский, Вспольный пер., 10",Баррикадная 8 мин. пешком Маяковская 10 мин. п...,1 пассажирский,...,Центральное,1.0,NaN,NaN,NaN,1 балкон,На улицу,1964,Есть,NaN
204996621,75000,https://www.cian.ru/rent/flat/204996621/,Комм. платежи включены (без счётчиков),37 м²,20 м²,"5,5 м²",4 из 8,"Москва, ЦАО, р-н Тверской, 4-я Тверская-Ямская...",Маяковская 5 мин. пешком,1 пассажирский,...,NaN,1.0,1.0,NaN,Есть,1 балкон,Во двор,NaN,Есть,"2,7 м"
211254455,79999,https://www.cian.ru/rent/flat/211254455/,Комм. платежи не включены,45 м²,33 м²,5 м²,6 из 8,"Москва, ЗАО, р-н Дорогомилово, Кутузовский про...",Деловой центр 7 мин. пешком Выставочная 7 мин....,1 пассажирский,...,NaN,NaN,NaN,Изолированная,NaN,NaN,NaN,NaN,Нет,0 м
214380553,80000,https://www.cian.ru/rent/flat/214380553/,Комм. платежи включены (без счётчиков),102 м²,65 м²,10 м²,4 из 5,"Москва, САО, р-н Савеловский, Петровско-Разумо...",Динамо 15 мин. пешком Савеловская 15 мин. пешк...,Нет,...,Центральное,1.0,1.0,NaN,Есть,2 балкона,Во двор,NaN,Нет,NaN
211793992,80000,https://www.cian.ru/rent/flat/211793992/,Комм. платежи включены (без счётчиков),86 м²,40 м²,16 м²,5 из 23,"Москва, СЗАО, р-н Южное Тушино, Сходненская ул...",Сходненская 10 мин. пешком,"2 пассажирских, 2 грузовых",...,NaN,NaN,NaN,Изолированная,NaN,1 лоджия,NaN,2010,Есть,NaN
211696644,83000,https://www.cian.ru/rent/flat/211696644/,Комм. платежи включены (без счётчиков),67 м²,NaN,7 м²,5 из 5,"Москва, СЗАО, р-н Щукино, ул. Маршала Бирюзова...",Октябрьское поле 10 мин. пешком,Нет,...,Центральное,1.0,NaN,NaN,NaN,NaN,На улицу и двор,1951,NaN,NaN
210572665,85000,https://www.cian.ru/rent/flat/210572665/,Комм. платежи включены (без счётчиков),105 м²,68 м²,14 м²,3 из 33,"Москва, ЗАО, р-н Тропарево-Никулино, просп. Ве...",Юго-Западная 1 мин. пешком Проспект Вернадског...,"2 пассажирских, 2 грузовых",...,NaN,1.0,1.0,Изолированная,Есть,1 лоджия,NaN,2008,Есть,NaN
213269803,86000,https://www.cian.ru/rent/flat/213269803/,Комм. платежи включены (без счётчиков),48 м²,34 м²,5 м²,14 из 14,"Москва, ЦАО, р-н Якиманка, ул. Большая Полянка...",Полянка 4 мин. пешком Добрынинская 7 мин. пешк...,2 пассажирских,...,NaN,1.0,1.0,NaN,Есть,2 лоджии,Во двор,1971,Есть,"2,8 м"


In [128]:
flat_df[['Год постройки', 'Построен', 'href']]

,Год постройки,Построен,href
207924378,2016,NaN,https://www.cian.ru/rent/flat/207924378/
214537449,2016,NaN,https://www.cian.ru/rent/flat/214537449/
186778851,2016,NaN,https://www.cian.ru/rent/flat/186778851/
215078648,2016,NaN,https://www.cian.ru/rent/flat/215078648/
151978998,NaN,2011,https://www.cian.ru/rent/flat/151978998/
214625671,2016,NaN,https://www.cian.ru/rent/flat/214625671/
207957981,2015,NaN,https://www.cian.ru/rent/flat/207957981/
214092509,NaN,NaN,https://www.cian.ru/rent/flat/214092509/
215229448,NaN,NaN,https://www.cian.ru/rent/flat/215229448/
208582656,2016,NaN,https://www.cian.ru/rent/flat/208582656/


In [129]:
flat_df.loc['204575572', :]

price                                                                                                                      220000
href                                                                                     https://www.cian.ru/rent/flat/204575572/
more_price_info                                                                            Комм. платежи включены (без счётчиков)
Общая                                                                                                                      171 м²
Жилая                                                                                                                      119 м²
Кухня                                                                                                                       15 м²
Этаж                                                                                                                     27 из 32
address                                                                         Москва, ЮЗ

In [86]:
flat_df

,price,href,more_price_info,Общая,Жилая,Кухня,Этаж,city,district,area,...,main_description,Можно с детьми,Можно с животными,Планировка,Построен,Тип перекрытий,Подъезды,Отопление,Аварийность,Балкон/лоджия
207924378,200000,https://www.cian.ru/rent/flat/207924378/,Комм. платежи включены (без счётчиков),75 м²,60 м²,10 м²,13 из 85,Москва,ЦАО,р-н Пресненский,...,Лот 013. О нас: ТОЛЬКО РЕАЛЬНО СУЩЕСТВУЮЩИЕ ОБ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
214537449,250000,https://www.cian.ru/rent/flat/214537449/,Комм. платежи включены (без счётчиков),82 м²,72 м²,5 м²,32 из 68,Москва,ЦАО,р-н Пресненский,...,Лот 708. Агентам солидный бонус. БАШНЯ ОКО. Ую...,1.0,1.0,Изолированная,NaN,NaN,NaN,NaN,NaN,NaN
186778851,199900,https://www.cian.ru/rent/flat/186778851/,Комм. платежи включены (без счётчиков),80 м²,70 м²,NaN,15 из 85,Москва,ЦАО,р-н Пресненский,...,СУПЕР-ЦЕНА! ОПЕРАТИВНЫЙ ПОКАЗ! ЗВОНИТЕ!БЕЗ КОМ...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215078648,200000,https://www.cian.ru/rent/flat/215078648/,Комм. платежи включены (без счётчиков),65 м²,50 м²,5 м²,13 из 85,Москва,ЦАО,р-н Пресненский,...,Апартаменты с отделкой в башне ОКО. 2-х комнат...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151978998,550000,https://www.cian.ru/rent/flat/151978998/,Комм. платежи включены (без счётчиков),193 м²,160 м²,10 м²,58 из 62,Москва,ЦАО,р-н Пресненский,...,Собственник сдает апартаменты в аренду по прям...,1.0,1.0,NaN,2011,NaN,NaN,NaN,NaN,NaN
214625671,300000,https://www.cian.ru/rent/flat/214625671/,Комм. платежи включены (без счётчиков),90 м²,75 м²,5 м²,38 из 85,Москва,ЦАО,р-н Пресненский,...,О нас: ТОЛЬКО РЕАЛЬНО СУЩЕСТВУЮЩИЕ ОБЪЕКТЫ!!! ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207957981,500000,https://www.cian.ru/rent/flat/207957981/,Комм. платежи включены (без счётчиков),220 м²,150 м²,20 м²,1 из 3,Москва,САО,р-н Хорошевский,...,МАКСИМАЛЬНЫЙ БОНУС АГЕНТУ! Предлагается таунха...,1.0,1.0,Изолированная,NaN,NaN,NaN,NaN,NaN,NaN
214092509,450000,https://www.cian.ru/rent/flat/214092509/,Комм. платежи включены (без счётчиков),190 м²,160 м²,10 м²,36 из 75,Москва,ЦАО,р-н Пресненский,...,О нас: ТОЛЬКО РЕАЛЬНО СУЩЕСТВУЮЩИЕ ОБЪЕКТЫ!!! ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215229448,500000,https://www.cian.ru/rent/flat/215229448/,Комм. платежи включены (без счётчиков),190 м²,160 м²,10 м²,21 из 75,Москва,ЦАО,р-н Пресненский,...,О нас: ТОЛЬКО РЕАЛЬНО СУЩЕСТВУЮЩИЕ ОБЪЕКТЫ!!! ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208582656,450000,https://www.cian.ru/rent/flat/208582656/,Комм. платежи включены (без счётчиков),190 м²,160 м²,10 м²,35 из 85,Москва,ЦАО,р-н Пресненский,...,Лот 035. О нас: ТОЛЬКО РЕАЛЬНО СУЩЕСТВУЮЩИЕ ОБ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
soup = BeautifulSoup(requests.get('https://www.cian.ru/rent/flat/207924378/').text)

'Лот 013. О нас: ТОЛЬКО РЕАЛЬНО СУЩЕСТВУЮЩИЕ ОБЪЕКТЫ!!!  ИХ МОЖНО ПОСМОТРЕТЬ СЕГОДНЯ!!! Мы в сити 24/7. Оперативный показ в любое удобное время. English  WhatsApp, Telegram, Viber. Агентам солидный бонус.Описание объекта: БАШНЯ Око. Уютная стандартная отделка. Апартаменты полностью укомплектованы мебелью и всей необходимой техникой бытовой и кухонной техникой. Функциональная планировка: Кухня-студия, совмещенная с просторной гостиной, спальня с собственной ванной и сан.узлом (сан.узлы так же полностью укомплектованы (См. фото)), просторный холл, постирочная и сушильная зона. Панорамное остекление по всему периметру апартаментов. Видовые характеристики: футуристический вид на город. В ночное время огни небоскребов никого не оставят равнодушными. Москва Сити - это Москва будущего, строящийся международный деловой квартал из ультрасовременных небоскрёбов. Уникальная для России и Восточной Европы зона деловой активности объединяет в себе апартаменты для жилья, офисные здания, многочисленны

In [153]:
''.join(re.findall(r'\d+', '23 267 предложений\xa0отсортированы\xa0По умолчанию'))

'23267'

In [26]:
%%time

for p in range(1,11):

    
    

    #По каждой квартире получаем основную информацию
    for room in room_url_list:
        #Записываем id квартиры и ссылку на нее
        room_id = re.search(r'[0-9]+',room.attrs['href']).group()
        flat_df.loc[room_id, 'ref'] = room.attrs['href'] 

        flat_html = requests.get(room.attrs['href'], headers=headers)
        bs_flat = BeautifulSoup(flat_html.text)
        print(flat_html)
        flat_df.loc[room_id, 'price'] = get_price(bs_flat)
        flat_df.loc[room_id, 'price_info'] = get_more_price_info(bs_flat)
        
        main_info = get_main_info(bs_flat)
        for main_info_col in main_info:
            flat_df.loc[room_id, main_info_col] = main_info[main_info_col]
            
        #address = get_address(bs_flat)
        #for add_part in address:
        #   flat_df.loc[room_id, add_part] = address[add_part]
        flat_df.loc[room_id, 'address'] = get_address_new(bs_flat)
        time.sleep(5)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

AttributeError: 'NoneType' object has no attribute 'attrs'